In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/HP3/LLM/SLM_Experiment/"

/content/drive/MyDrive/HP3/LLM/SLM_Experiment


# 1.Load Dataset Resume-JD

In [ ]:
import pandas as pd
import json
from pathlib import Path

def create_realtest_df(root_path):
    root = Path(root_path)

    # Load Matching_Result.json
    with open(root / "Matching_Result.json", "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []
    for item in data:
        cv_name = item["CV"] + ".txt"
        jd_name = item["JD"] + ".txt"
        score = item["Score"]

        cv_path = root / "CV" / cv_name
        jd_path = root / "JD" / jd_name

        # Đọc nội dung file CV và JD
        if cv_path.exists() and jd_path.exists():
            with open(cv_path, "r", encoding="utf-8") as f:
                cv_text = f.read().strip()
            with open(jd_path, "r", encoding="utf-8") as f:
                jd_text = f.read().strip()

            rows.append({
                "CV_Text": cv_text,
                "JD_Text": jd_text,
                "Score": score
            })
        else:
            print(f"⚠️ File missing: {cv_name} or {jd_name}")

    df_realtest = pd.DataFrame(rows)
    return df_realtest


df_real = create_realtest_df("../Data_Collector_11_9")

df_train = pd.read_csv("./resume_jd_kaggle_dataset/train.csv")
df_test = pd.read_csv("./resume_jd_kaggle_dataset/test.csv")
df_test

,job_description,resume,match_score,data_id
0,Product Manager needed with experience in Scru...,"Experienced professional skilled in Agile, Use...",8,id_6252
1,Product Manager needed with experience in Prod...,"Experienced professional skilled in life, Scru...",4,id_4684
2,Data Scientist needed with experience in Machi...,"Experienced professional skilled in night, alt...",4,id_1731
3,Data Analyst needed with experience in Tableau...,"Experienced professional skilled in book, SQL,...",4,id_4742
4,"ML Engineer needed with experience in Python, ...",Experienced professional skilled in Computer V...,8,id_4521
...,...,...,...,...
2995,"Data Analyst needed with experience in SQL, Ex...","Experienced professional skilled in SQL, Excel...",6,id_8014
2996,Product Manager needed with experience in Agil...,Experienced professional skilled in Stakeholde...,6,id_1074
2997,Product Manager needed with experience in Stak...,"Experienced professional skilled in might, fam...",4,id_3063
2998,ML Engineer needed with experience in Computer...,"Experienced professional skilled in Keras, com...",2,id_6487


# 2.Create Resume-JD Embedding

In [ ]:
# model_list = [
#     "sentence-transformers/all-MiniLM-L6-v2",
#     "sentence-transformers/paraphrase-MiniLM-L6-v2",

#     "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
#     "sentence-transformers/distiluse-base-multilingual-cased-v2",

#     "sentence-transformers/all-MiniLM-L12-v2",
#     "sentence-transformers/all-mpnet-base-v2",
#     "sentence-transformers/paraphrase-mpnet-base-v2",
#     "sentence-transformers/multi-qa-mpnet-base-dot-v1",

#     "sentence-transformers/stsb-xlm-r-multilingual",
#     "sentence-transformers/all-distilroberta-v1"
# ]

import os
from sentence_transformers import SentenceTransformer
import pandas as pd
from multiprocessing import Pool

# =============================
# Helper: encode single text
# =============================
def _encode_single(args):
    text, model = args
    return model.encode(text, convert_to_tensor=True)

# =============================
# Create embedding dataframe
# =============================
def create_bert_embeddings_df(input_df,
                              job_col="job_description",
                              resume_col="resume",
                              model_name="sentence-transformers/all-MiniLM-L6-v2",
                              # model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
                              # model_name="all-distilroberta-v1",
                              batch_size=32,
                              output_path=None,
                              n_cores=10):
    # Load model
    model = SentenceTransformer(model_name)

    # Convert columns to list
    jobs = input_df[job_col].astype(str).tolist()
    resumes = input_df[resume_col].astype(str).tolist()

    # Encode in batch (faster than multiprocessing for model.encode)
    job_embeds = model.encode(jobs, batch_size=batch_size, convert_to_tensor=True, show_progress_bar=True)
    resume_embeds = model.encode(resumes, batch_size=batch_size, convert_to_tensor=True, show_progress_bar=True)

    # Add back to dataframe
    df_output = input_df.copy()
    df_output["job_embedding"] = job_embeds.tolist()
    df_output["resume_embedding"] = resume_embeds.tolist()

    # Save if needed
    if output_path:
        df_output.to_pickle(output_path)  # pickle để giữ được tensor object

    return df_output

df_train_embed = create_bert_embeddings_df(df_train,
                                           job_col="job_description",
                                           resume_col="resume",
                                          #  model_name="sentence-transformers/all-MiniLM-L6-v2",
                                           batch_size=32,
                                           output_path="./res_output_seperation/train_with_embeddings.pkl",
                                           n_cores=10)
df_train_embed.to_csv("./res_output_seperation/df_train_embed.csv", index=False)

df_test_embed = create_bert_embeddings_df(df_test,
                                           job_col="job_description",
                                           resume_col="resume",
                                          #  model_name="sentence-transformers/all-MiniLM-L6-v2",
                                           batch_size=32,
                                           output_path="./res_output_seperation/test_with_embeddings.pkl",
                                           n_cores=10)
df_test_embed.to_csv("./res_output_seperation/df_test_embed.csv", index=False)

df_real_embed = create_bert_embeddings_df(df_real,
                                          job_col="JD_Text",
                                          resume_col="CV_Text",
                                          # model_name="sentence-transformers/all-MiniLM-L6-v2",
                                          batch_size=32,
                                          output_path="./res_output_seperation/87_pairs_with_embeddings.pkl",
                                          n_cores=10)
df_real_embed.to_csv("./res_output_seperation/df_test_embed.csv", index=False)

Batches:   0%|          | 0/219 [00:00<?, ?it/s]

Batches:   0%|          | 0/219 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import numpy as np
len(df_test_embed['job_embedding'][0:1].values[0])

# 3.Model training - Ridge Regression


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, RidgeCV, LassoCV, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# XGBoost (may require `pip install xgboost`)
try:
    from xgboost import XGBRegressor
except Exception as e:
    XGBRegressor = None
    print("⚠️ xgboost not available. Install with `pip install xgboost` to use XGBoost models.")

# -------------------------
# Utils
# -------------------------
def prepare_feature_matrix(df, emb_col1="job_embedding", emb_col2="resume_embedding"):
    """
    Trả về X: ghép 2 embedding (mỗi row là numpy array)
    """
    emb1 = np.stack(df[emb_col1].values)
    emb2 = np.stack(df[emb_col2].values)
    X = np.hstack([emb1, emb2])
    return X

def compute_accuracy(df, label_col="match_score", pred_col="predicted_match_score"):
    """
    Acc@1, Acc@2, Acc@3 (tỉ lệ |label - pred| <= k)
    df must contain label_col and pred_col (pred_col should be integer predictions)
    """
    acc1 = (np.abs(df[label_col] - df[pred_col]) <= 1).mean()
    acc2 = (np.abs(df[label_col] - df[pred_col]) <= 2).mean()
    acc3 = (np.abs(df[label_col] - df[pred_col]) <= 3).mean()
    return acc1, acc2, acc3

def plot_score_distribution(df, model_name, chart_output_folder, label_col="match_score", pred_col="predicted_match_score", suffix=""):
    os.makedirs(chart_output_folder, exist_ok=True)
    plt.figure(figsize=(10, 6))
    plt.hist(df[label_col], bins=10, alpha=0.6, label='Labels')
    plt.hist(df[pred_col], bins=10, alpha=0.6, label='Predicts')
    plt.title(f"Score Distribution - {model_name} {suffix}", fontsize=14)
    plt.xlabel("Score", fontsize=12)
    plt.ylabel("Count", fontsize=12)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    safe_name = model_name.replace('/', '_')
    save_path = os.path.join(chart_output_folder, f"{safe_name}{('_'+suffix) if suffix else ''}.png")
    plt.savefig(save_path)
    plt.close()
    print(f"✅ Saved chart to {save_path}")

# -------------------------
# Trainers for each method
# -------------------------
def train_and_eval(df_train,
                   df_test_embed,
                   method,
                   emb_col1="job_embedding",
                   emb_col2="resume_embedding",
                   label_col="match_score",
                   test_size=0.2,
                   random_state=42,
                   n_components=200,
                   chart_folder="./res_output/charts"):
    """
    Train on df_train (but still use train_test_split for internal validation).
    Then evaluate on both internal validation split (from train_test_split) and on df_test_embed (external).
    Returns detailed artifacts & metrics for both internal and external.
    """
    assert method in ["pca_ridge", "ridge", "pca_lasso", "pca_svr", "pca_randomforest", "pca_xgboost"], \
        f"Unknown method: {method}"

    # Prepare X_train_all, y_train_all
    X_all = prepare_feature_matrix(df_train, emb_col1, emb_col2)
    y_all = df_train[label_col].values.astype(float)

    # Internal split (used for quick validation)
    X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, test_size=test_size, random_state=random_state)

    # External test features/labels
    X_external = prepare_feature_matrix(df_test_embed, emb_col1, emb_col2)
    y_external = df_test_embed[label_col].values.astype(float)

    # Scaler fitted on internal train
    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)
    X_val_std = scaler.transform(X_val)
    X_external_std = scaler.transform(X_external)

    pca = None
    if method.startswith("pca_"):
        # apply PCA fitted on internal train
        pca = PCA(n_components=n_components)
        X_train_proc = pca.fit_transform(X_train_std)
        X_val_proc = pca.transform(X_val_std)
        X_external_proc = pca.transform(X_external_std)
        print(f"🔷 [{method}] PCA n_components={n_components} explained_variance_sum={pca.explained_variance_ratio_.sum():.4f}")
    else:
        X_train_proc = X_train_std
        X_val_proc = X_val_std
        X_external_proc = X_external_std

    model = None
    # ---------- Ridge family ----------
    if method in ("pca_ridge", "ridge"):
        alphas = np.logspace(-3, 3, 13)
        ridge_cv = RidgeCV(alphas=alphas, scoring="neg_mean_squared_error", cv=5)
        ridge_cv.fit(X_train_proc, y_train)
        best_alpha = ridge_cv.alpha_
        model = Ridge(alpha=best_alpha, random_state=random_state)
        model.fit(X_train_proc, y_train)
        print(f"🔧 [{method}] Ridge trained. best_alpha={best_alpha}")
    # ---------- Lasso ----------
    elif method == "pca_lasso":
        lasso_cv = LassoCV(cv=5, n_alphas=50, random_state=random_state, max_iter=5000)
        lasso_cv.fit(X_train_proc, y_train)
        best_alpha = lasso_cv.alpha_
        model = Lasso(alpha=best_alpha, max_iter=10000)
        model.fit(X_train_proc, y_train)
        print(f"🔧 [{method}] Lasso trained. best_alpha={best_alpha}")
    # ---------- SVR ----------
    elif method == "pca_svr":
        model = SVR()
        model.fit(X_train_proc, y_train)
        print(f"🔧 [{method}] SVR trained.")
    # ---------- RandomForest ----------
    elif method == "pca_randomforest":
        rf = RandomForestRegressor(n_estimators=200, random_state=random_state, n_jobs=-1)
        rf.fit(X_train_proc, y_train)
        model = rf
        print(f"🔧 [{method}] RandomForest trained.")
    # ---------- XGBoost ----------
    elif method == "pca_xgboost":
        if XGBRegressor is None:
            raise ImportError("xgboost not installed. Please `pip install xgboost` to use pca_xgboost.")
        xgb = XGBRegressor(n_estimators=200, verbosity=0, random_state=random_state, n_jobs=-1)
        xgb.fit(X_train_proc, y_train)
        model = xgb
        print(f"🔧 [{method}] XGBoost trained.")
    else:
        raise NotImplementedError(method)

    # -------------------------
    # Internal Validation Predictions (from train_test_split)
    # -------------------------
    preds_val = model.predict(X_val_proc)
    preds_val_rounded = preds_val.round().astype(int)
    preds_val_rounded = np.clip(preds_val_rounded, 0, 10)

    # df_internal = pd.DataFrame({
    #     label_col: y_val,
    #     "predicted_match_score": preds_val_rounded
    # })

    df_internal = pd.DataFrame({
        label_col: y_val,
        "predicted_match_score": preds_val_rounded
    })
    df_internal["MAE"] = (df_internal[label_col] - df_internal["predicted_match_score"]).abs()


    acc1_int, acc2_int, acc3_int = compute_accuracy(df_internal, label_col=label_col, pred_col="predicted_match_score")
    # mse_int = mean_squared_error(y_val, preds_val)
    # r2_int = r2_score(y_val, preds_val)

    print(f"📊 [{method}] Internal Val Split → Acc@1={acc1_int:.4f} Acc@2={acc2_int:.4f} Acc@3={acc3_int:.4f}  MAE={df_internal["MAE"].mean():.4f}")

    # save internal chart
    try:
        plot_score_distribution(df_internal, method, chart_folder, label_col=label_col, pred_col="predicted_match_score", suffix="internal")
    except Exception as e:
        print(f"⚠️ Failed to plot internal distribution for {method}: {e}")

    # -------------------------
    # External Test Predictions (df_test_embed)
    # -------------------------
    preds_ext = model.predict(X_external_proc)
    preds_ext_rounded = preds_ext.round().astype(int)
    preds_ext_rounded = np.clip(preds_ext_rounded, 0, 10)

    # df_external = pd.DataFrame({
    #     label_col: y_external,
    #     "predicted_match_score": preds_ext_rounded
    # })
    df_external = pd.DataFrame({
        label_col: y_external,
        "predicted_match_score": preds_ext_rounded
    })
    df_external["MAE"] = (df_external[label_col] - df_external["predicted_match_score"]).abs()


    acc1_ext, acc2_ext, acc3_ext = compute_accuracy(df_external, label_col=label_col, pred_col="predicted_match_score")
    mse_ext = mean_squared_error(y_external, preds_ext)
    r2_ext = r2_score(y_external, preds_ext)

    print(f"📊 [{method}] External Val Split → Acc@1={acc1_ext:.4f}, Acc@2={acc2_ext:.4f}, Acc@3={acc3_ext:.4f}, MAE={df_external['MAE'].mean():.4f}")

    # save external chart
    try:
        plot_score_distribution(df_external, method, chart_folder, label_col=label_col, pred_col="predicted_match_score", suffix="external")
    except Exception as e:
        print(f"⚠️ Failed to plot external distribution for {method}: {e}")

    # return artifacts
    results = {
        "model": model,
        "scaler": scaler,
        "pca": pca,
        "df_internal_out": df_internal,
        "df_external_out": df_external,
        "metrics_internal": {
            "Acc@1": acc1_int, "Acc@2": acc2_int, "Acc@3": acc3_int,
            # "MSE": mse_int, "R2": r2_int
            "MAE": df_internal["MAE"].mean()
        },
        "metrics_external": {
            "Acc@1": acc1_ext, "Acc@2": acc2_ext, "Acc@3": acc3_ext,
            # "MSE": mse_ext, "R2": r2_ext
            "MAE": df_internal["MAE"].mean()
        }
    }
    return results

# -------------------------
# Runner: chạy nhiều phương pháp và lưu summary
# -------------------------
def run_pipeline_all(df_train_embed,
                     df_test_embed,
                     method_list=None,
                     emb_col1="job_embedding",
                     emb_col2="resume_embedding",
                     label_col="match_score",
                     output_csv="./res_output/metrics_summary.csv",
                     chart_folder="./res_output/charts",
                     test_size=0.2,
                     random_state=42,
                     n_components=200):
    """
    Runs multiple methods, trains on df_train_embed (using internal split for validation),
    evaluates on df_test_embed (external), and writes a detailed CSV with both internal and external metrics.
    """
    if method_list is None:
        method_list = ["pca_ridge", "ridge", "pca_lasso", "pca_svr", "pca_randomforest", "pca_xgboost"]

    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    os.makedirs(chart_folder, exist_ok=True)

    records = []
    for method in method_list:
        print("\n" + "="*60)
        try:
            res = train_and_eval(df_train_embed,
                                 df_test_embed,
                                 method=method,
                                 emb_col1=emb_col1,
                                 emb_col2=emb_col2,
                                 label_col=label_col,
                                 test_size=test_size,
                                 random_state=random_state,
                                 n_components=n_components,
                                 chart_folder=chart_folder)

            m_int = res["metrics_internal"]
            m_ext = res["metrics_external"]

            # 👉 Tính MAE trung bình
            mae_int = res["df_internal_out"]["MAE"].mean()
            mae_ext = res["df_external_out"]["MAE"].mean()

            records.append({
                "method": method,
                "Acc@1_internal": m_int["Acc@1"],
                "Acc@2_internal": m_int["Acc@2"],
                "Acc@3_internal": m_int["Acc@3"],
                "MAE_internal": mae_int,
                "Acc@1_external": m_ext["Acc@1"],
                "Acc@2_external": m_ext["Acc@2"],
                "Acc@3_external": m_ext["Acc@3"],
                "MAE_external": mae_ext
            })

        except Exception as e:
            print(f"❌ Method {method} failed: {e}")
            records.append({
                "method": method,
                "Acc@1_internal": np.nan,
                "Acc@2_internal": np.nan,
                "Acc@3_internal": np.nan,
                "MAE_internal": np.nan,
                "Acc@1_external": np.nan,
                "Acc@2_external": np.nan,
                "Acc@3_external": np.nan,
                "MAE_external": np.nan
            })

    df_summary = pd.DataFrame(records)
    df_summary.to_csv(output_csv, index=False)
    print(f"\n✅ Saved metrics summary (with MAE only) to {output_csv}")
    return df_summary

method_list = ["pca_ridge","ridge","pca_lasso","pca_svr"]
df_metrics = run_pipeline_all(df_train_embed, df_test_embed, method_list=method_list)


🔷 [pca_ridge] PCA n_components=200 explained_variance_sum=0.8469
🔧 [pca_ridge] Ridge trained. best_alpha=1000.0
📊 [pca_ridge] Internal Val Split → Acc@1=0.6257 Acc@2=0.8579 Acc@3=0.9629  MAE=1.3214
✅ Saved chart to ./res_output/charts/pca_ridge_internal.png
📊 [pca_ridge] External Val Split → Acc@1=0.6353, Acc@2=0.8630, Acc@3=0.9577, MAE=1.3170
✅ Saved chart to ./res_output/charts/pca_ridge_external.png

🔧 [ridge] Ridge trained. best_alpha=316.22776601683796
📊 [ridge] Internal Val Split → Acc@1=0.6857 Acc@2=0.8914 Acc@3=0.9671  MAE=1.1750
✅ Saved chart to ./res_output/charts/ridge_internal.png
📊 [ridge] External Val Split → Acc@1=0.6763, Acc@2=0.8900, Acc@3=0.9710, MAE=1.1887
✅ Saved chart to ./res_output/charts/ridge_external.png

🔷 [pca_lasso] PCA n_components=200 explained_variance_sum=0.8469
🔧 [pca_lasso] Lasso trained. best_alpha=0.007800077553864851
📊 [pca_lasso] Internal Val Split → Acc@1=0.6286 Acc@2=0.8614 Acc@3=0.9614  MAE=1.3193
✅ Saved chart to ./res_output/charts/pca_lasso

# 4.Evaluate on test set

In [ ]:
df_real_embed = df_real_embed.rename(columns={'Score': 'match_score'})

df_real_embed

,CV_Text,JD_Text,match_score,job_embedding,resume_embedding
0,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,Scientists ZAP advise business stakeholders on...,6,"[-0.057775355875492096, -0.003755283309146762,...","[-0.09361451864242554, -0.0037099875044077635,..."
1,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,AI Engineer (AI Solution Team)\nFPT Smart Clou...,6,"[-0.028071988373994827, -0.04387618228793144, ...","[-0.09361451864242554, -0.0037099875044077635,..."
2,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,Position summary\nIndustry: IT & Internet\nJob...,3,"[-0.047819338738918304, -0.01788148283958435, ...","[-0.09361451864242554, -0.0037099875044077635,..."
3,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,Kỹ sư Dữ liệu lớn (Big Data Engineer)\n1. MÔ T...,3,"[-0.11163052171468735, 0.04298354685306549, 0....","[-0.09361451864242554, -0.0037099875044077635,..."
4,TRẦN HOÀN ĐỨC DUY\n& 0888 193 215 :unselected:...,Job description\nWe are looking for a Data Eng...,3,"[-0.0251266248524189, 0.03949412703514099, -0....","[-0.09361451864242554, -0.0037099875044077635,..."
...,...,...,...,...,...
82,"NGUYEN UT THIEN BACKEND NODEJS\nMAY 22, 1998\n...",Job description\nAbout the job\n\nFast and inn...,10,"[-0.055324096232652664, -0.06262123584747314, ...","[-0.022309456020593643, 0.039068371057510376, ..."
83,"NGUYEN UT THIEN BACKEND NODEJS\nMAY 22, 1998\n...",Mô tả công việc\n- Chào mời & hướng dẫn chủ cử...,0,"[-0.08064166456460953, 0.011131036095321178, 0...","[-0.022309456020593643, 0.039068371057510376, ..."
84,LIÊN HỆ\n0767506071\ntophuocthanh2@gmail.com\n...,Mô tả công việc\n- Chào mời & hướng dẫn chủ cử...,10,"[-0.08064166456460953, 0.011131036095321178, 0...","[-0.06816567480564117, 0.012177761644124985, -..."
85,Ninh Hoang Khai FRESHER FRONTEND DEVELOPER\nnh...,Job description\nAbout the job\n\nFast and inn...,8,"[-0.055324096232652664, -0.06262123584747314, ...","[-0.04103987291455269, 0.009629149921238422, 0..."


In [ ]:
df_metrics = run_pipeline_all(df_train_embed, df_real_embed, method_list=method_list)


🔷 [pca_ridge] PCA n_components=200 explained_variance_sum=0.8469
🔧 [pca_ridge] Ridge trained. best_alpha=1000.0
📊 [pca_ridge] Internal Val Split → Acc@1=0.6343 Acc@2=0.8521 Acc@3=0.9636  MAE=1.3264
✅ Saved chart to ./res_output/charts/pca_ridge_internal.png
📊 [pca_ridge] External Val Split → Acc@1=0.3908, Acc@2=0.5517, Acc@3=0.7126, MAE=2.8046
✅ Saved chart to ./res_output/charts/pca_ridge_external.png

🔧 [ridge] Ridge trained. best_alpha=316.22776601683796
📊 [ridge] Internal Val Split → Acc@1=0.6857 Acc@2=0.8914 Acc@3=0.9671  MAE=1.1750
✅ Saved chart to ./res_output/charts/ridge_internal.png
📊 [ridge] External Val Split → Acc@1=0.2989, Acc@2=0.4828, Acc@3=0.6092, MAE=3.4253
✅ Saved chart to ./res_output/charts/ridge_external.png

🔷 [pca_lasso] PCA n_components=200 explained_variance_sum=0.8470
🔧 [pca_lasso] Lasso trained. best_alpha=0.0067744560705940695
📊 [pca_lasso] Internal Val Split → Acc@1=0.6336 Acc@2=0.8579 Acc@3=0.9586  MAE=1.3229
✅ Saved chart to ./res_output/charts/pca_lass

In [ ]:
# paraphrase-multilingual-MiniLM-L12-v2
df_metrics = run_pipeline_all(df_train_embed, df_real_embed, method_list=method_list)


🔷 [pca_ridge] PCA n_components=200 explained_variance_sum=0.9008
🔧 [pca_ridge] Ridge trained. best_alpha=1000.0
📊 [pca_ridge] Internal Val Split → Acc@1=0.5821 Acc@2=0.8364 Acc@3=0.9486  MSE=3.2563 R2=0.4266
✅ Saved chart to ./res_output/charts/pca_ridge_internal.png
📊 [pca_ridge] External Test Set → Acc@1=0.3218 Acc@2=0.4253 Acc@3=0.5862  MSE=21.8123 R2=-1.7627
✅ Saved chart to ./res_output/charts/pca_ridge_external.png

🔧 [ridge] Ridge trained. best_alpha=31.622776601683793
📊 [ridge] Internal Val Split → Acc@1=0.6857 Acc@2=0.9029 Acc@3=0.9714  MSE=2.3570 R2=0.5850
✅ Saved chart to ./res_output/charts/ridge_internal.png
📊 [ridge] External Test Set → Acc@1=0.2069 Acc@2=0.3793 Acc@3=0.5402  MSE=30.5067 R2=-2.8639
✅ Saved chart to ./res_output/charts/ridge_external.png

🔷 [pca_lasso] PCA n_components=200 explained_variance_sum=0.9008
🔧 [pca_lasso] Lasso trained. best_alpha=0.00907098910004918
📊 [pca_lasso] Internal Val Split → Acc@1=0.5943 Acc@2=0.8514 Acc@3=0.9521  MSE=3.1666 R2=0.4424

In [ ]:
df_metrics = run_pipeline_all(df_train_embed, df_real_embed, method_list=method_list)


🔷 [pca_ridge] PCA n_components=200 explained_variance_sum=0.8412
🔧 [pca_ridge] Ridge trained. best_alpha=1000.0
📊 [pca_ridge] Internal Val Split → Acc@1=0.5971 Acc@2=0.8571 Acc@3=0.9593  MSE=3.0675 R2=0.4599
✅ Saved chart to ./res_output/charts/pca_ridge_internal.png
📊 [pca_ridge] External Test Set → Acc@1=0.3448 Acc@2=0.5632 Acc@3=0.7241  MSE=9.5771 R2=-0.2130
✅ Saved chart to ./res_output/charts/pca_ridge_external.png

🔧 [ridge] Ridge trained. best_alpha=316.22776601683796
📊 [ridge] Internal Val Split → Acc@1=0.6743 Acc@2=0.9043 Acc@3=0.9657  MSE=2.4139 R2=0.5750
✅ Saved chart to ./res_output/charts/ridge_internal.png
📊 [ridge] External Test Set → Acc@1=0.2874 Acc@2=0.4943 Acc@3=0.6897  MSE=11.8963 R2=-0.5067
✅ Saved chart to ./res_output/charts/ridge_external.png

🔷 [pca_lasso] PCA n_components=200 explained_variance_sum=0.8411
🔧 [pca_lasso] Lasso trained. best_alpha=0.018791992776075492
📊 [pca_lasso] Internal Val Split → Acc@1=0.5964 Acc@2=0.8486 Acc@3=0.9607  MSE=3.0826 R2=0.4572